# importing Dependencies

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

Reading dataset

In [3]:
dataset = pd.read_csv("../input/students-performance-in-exams/StudentsPerformance.csv")
X = dataset.drop(['math score', 'lunch','race/ethnicity'],1)
y = dataset['math score'].values
y = y.reshape(len(y),1)


# Data preprocessing

Handling catogrical variables using colomun transformer, one hot encoding

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
ct = ColumnTransformer(transformers = [("encoder1", OneHotEncoder(),['parental level of education']), ("encoder2", OneHotEncoder(),['gender']),("encoder3", OneHotEncoder(),['test preparation course']), ("encoder4", OneHotEncoder(),['test preparation course'])], remainder ="passthrough")
X = np.array(ct.fit_transform(X))


[('encoder1', OneHotEncoder(), ['parental level of education']),
 ('encoder2', OneHotEncoder(), ['gender']),
 ('encoder3', OneHotEncoder(), ['test preparation course']),
 ('encoder4', OneHotEncoder(), ['test preparation course']),
 ('remainder', 'passthrough', [3, 4])]

spliting the dataset using train_test_split

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 5)


scaling the dataset using Standard scaller

In [6]:
from sklearn.preprocessing import StandardScaler
scx = StandardScaler()
scy = StandardScaler()
X_train[:,:12]=scx.fit_transform(X_train[:,:12])
X_test[:,:12]= scx.transform(X_test[:,:12])
y_train = scy.fit_transform(y_train)

# Model building

using svr for regression

In [7]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train.ravel())

SVR()

# Making prediction

In [8]:
y_pred = scy.inverse_transform(regressor.predict(X_test))
np.set_printoptions(precision = 2)
print(y_pred)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

[59.48 78.6  61.85 50.44 57.49 65.74 72.84 58.36 50.83 58.83 66.46 73.98
 67.26 86.76 76.12 51.73 71.66 54.   50.4  68.39 70.96 71.5  51.78 69.55
 63.19 55.13 53.24 56.83 59.88 60.71 74.24 74.57 63.57 57.9  85.43 54.16
 81.21 41.11 82.57 79.18 79.5  41.54 89.13 56.21 65.6  47.3  78.47 50.2
 48.71 77.74 67.56 48.94 81.76 57.21 75.38 77.54 81.19 63.76 47.16 51.95
 44.15 78.63 37.22 63.7  79.32 73.33 60.25 41.87 60.23 56.11 77.23 75.34
 69.57 55.53 52.65 50.57 68.2  68.65 43.15 70.78 52.27 54.97 67.39 57.75
 79.17 67.29 63.54 72.29 83.34 80.21 69.62 55.36 63.3  60.07 75.26 58.68
 42.74 83.2  51.63 74.84 90.36 60.04 73.69 71.93 74.13 56.81 56.46 76.71
 70.64 56.42 51.68 56.14 70.23 82.2  69.46 61.9  78.29 85.47 84.66 86.6
 66.46 56.83 72.22 70.66 74.39 80.92 68.37 71.59 26.35 75.77 50.17 72.6
 67.88 59.52 87.85 63.88 82.65 67.29 53.05 65.27 46.   46.58 52.5  74.75
 69.95 78.06 69.99 80.15 74.17 50.82 72.72 66.72 87.99 65.52 84.39 68.4
 64.52 67.5  79.38 52.17 75.38 54.19 77.36 65.87 80.21 

Preformance measurement using r2socre

In [9]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.7732886671259396

cross validation of model using srocc_val_score

In [10]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor, X = X_test, y= y_test.ravel(), cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard davition: {:.2f}%".format(accuracies.std()*100))


Accuracy: 51.58 %
Standard davition: 5.21%


# Hyperparameter tuning
 Using Grid_search_cv

In [11]:
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']},
              {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,0.9,]}]
grid_search = GridSearchCV(estimator = regressor,
                          param_grid = parameters,
                          scoring = 'r2',
                           n_jobs = -1)
grid_search.fit(X_train, y_train.ravel())
best_accuracy = grid_search.best_score_
best_prameter = grid_search.best_params_
print("best accuracy: {:.2f}%". format(best_accuracy*100))
print("best parameter ", best_prameter)

best accuracy: 83.92%
best parameter  {'C': 0.75, 'kernel': 'linear'}
